In [1]:
import numpy as np
import pandas as pd
import numpy.random as rgt
from scipy.stats import norm
from scipy.optimize import fsolve
from selectinf.base import selected_targets
from selectinf.QR_lasso import QR_lasso
from selectinf.randomization import randomization
from selectinf.approx_reference import approximate_grid_inference
from selectinf.exact_reference import exact_grid_inference
from selectinf.regreg_QR.QR_population import *

# set random seed
np.random.seed(2023)
np.set_printoptions(threshold=np.inf)

In [2]:
# generate covariance matrix
def cov_generate(std, rho):
    p = len(std)
    R = np.abs(np.subtract.outer(np.arange(p), np.arange(p)))
    return np.outer(std, std) * (rho ** R)

# estimate AR covariance matrix
def cov_estmate(X, t = 100):
    ARrho = []
    for s in np.random.sample(t):
        Xr = X[int(s * n)]
        ARrho.append(np.corrcoef(Xr[1:], Xr[:-1])[0, 1])
    ARrho = np.mean(ARrho)
    ARcov = ARrho ** (np.abs(np.subtract.outer(np.arange(p), np.arange(p))))
    return ARcov

# Homogeneous model

$$
    y_i= \boldsymbol{x}_i ^ \top \boldsymbol{\beta}^* + \varepsilon_i-F_{\varepsilon_i}^{-1}(\tau)
$$

For our targets, we consider two different targets for implement purpose:
1. the first target solves population smoothed quantile loss function
2. the second target solves check loss function


Next, we focus on the details about first target.
<img style="float: left;" src="note1.png" width="750"/>

In [3]:
# model setting
num_set = []
coverage_exact1 = []
coverage_exact2 = []
coverage_approx1 = []
coverage_approx2 = []
length_exact = []
length_approx = []

n, p = 800, 200
mu, Sig = np.zeros(p), cov_generate(np.ones(p), 0.5)
beta = np.zeros(p)
beta[0:5] = 1
tau = 0.75

In [4]:
reps = 500
for i in range(reps):
    print(i)
    # generate sample
    X = rgt.multivariate_normal(mean=mu, cov=Sig, size=n)
    Y = X.dot(beta) + rgt.normal(loc=0, scale=1, size=n) - norm.ppf(tau, loc=0, scale=1)

    # inference
    randomizer = randomization.isotropic_gaussian(shape=(p,), scale=0.75*(1/np.sqrt(n)))
    conv = QR_lasso(X,
                    Y,
                    tau=tau,
                    randomizer=randomizer)
    conv.fit()
    conv.setup_inference()
    query_spec = conv.specification
    target_spec, bw = selected_targets(X,
                                       Y,
                                       tau=tau,
                                       solution=conv.observed_soln)

    # nonzero set of penalized estimator
    nonzero_set = np.nonzero(conv.observed_soln)[0]
    num_set.append(nonzero_set.size)
    print(conv.observed_soln[nonzero_set])
    print(target_spec.observed_target)

    # beta target    
    value = fsolve(find_root, 0, args=(tau, bw, 'Gaussian'))
    beta_target1 = np.linalg.pinv(X[:, nonzero_set]).dot(X.dot(beta) - value)
    beta_target2 = np.linalg.pinv(X[:, nonzero_set]).dot(X.dot(beta))
    print(beta_target1)
    print(beta_target2)

    # exact pivot
    exact_grid_inf = exact_grid_inference(query_spec, target_spec)
    lci, uci = exact_grid_inf._intervals(level=0.90)
    length = uci - lci
    coverage1 = (lci < beta_target1) * (uci > beta_target1)
    coverage2 = (lci < beta_target2) * (uci > beta_target2)
    coverage_exact1.append(np.mean(coverage1))
    coverage_exact2.append(np.mean(coverage2))
    length_exact.append(np.mean(length))

    # approximate pivot
    approximate_grid_inf = approximate_grid_inference(query_spec, target_spec)
    lci, uci = approximate_grid_inf._intervals(level=0.90)
    length = uci - lci
    coverage1 = (lci < beta_target1) * (uci > beta_target1)
    coverage2 = (lci < beta_target2) * (uci > beta_target2)
    coverage_approx1.append(np.mean(coverage1))
    coverage_approx2.append(np.mean(coverage2))
    length_approx.append(np.mean(length))


0
[0.771442   0.74713877 1.07388299 0.74631389 0.83287475]
[1.05837666 0.96434992 1.00813981 1.04135739 0.97669037]
[1.00052753 1.0001328  0.99957062 0.99965612 0.9998223 ]
[1. 1. 1. 1. 1.]
1
[ 0.82814313  1.03301844  0.88641203  0.72890543  0.92291869 -0.01239106]
[ 1.01794667  1.03965847  0.91568314  0.96665977  1.12240451 -0.02425444]
[9.99785121e-01 9.99800350e-01 9.99853583e-01 1.00049648e+00
 9.99553016e-01 2.09973646e-04]
[1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 2.56739074e-16]
2
[0.44110041 1.13334439 0.92128023 0.71229868 0.96168491 0.0756806
 0.02903409]
[0.91527746 1.00336461 1.00868571 0.95898846 1.02623068 0.01766011
 0.07556989]
[ 9.99483685e-01  1.00028956e+00  9.99855886e-01  1.00029171e+00
  1.00029449e+00  3.77758624e-04 -7.66717158e-04]
[ 1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
  1.00000000e+00 -1.38777878e-16  1.97758476e-16]
3
[0.77023917 0.91145838 0.66825379 1.26040377 0.58658546 0.0369618 ]
[1.00280018 1

[ 1.00042141e+00  9.99167221e-01  9.99550903e-01  9.99987108e-01
  1.00009006e+00 -1.92644507e-04 -2.08307620e-04]
[ 1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
  1.00000000e+00  1.78676518e-16 -1.24900090e-16]
26
[ 0.68616356  1.03740034  0.96098482  0.75839717  0.82315299 -0.04094918]
[ 0.96964118  0.97439583  0.9789171   1.05944644  0.96727034 -0.09180006]
[ 1.00024245e+00  1.00027195e+00  1.00009991e+00  9.99796021e-01
  1.00030798e+00 -6.70048856e-04]
[ 1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
  1.00000000e+00 -4.71844785e-16]
27
[ 5.59506748e-01  1.18386318e+00  9.01990488e-01  1.00649352e+00
  6.32566982e-01  2.48177373e-03 -9.82059983e-04  1.05568504e-02]
[0.98651506 1.05536367 1.06998632 0.96276919 0.91104018 0.00535207
 0.00480274 0.05266259]
[ 9.99969277e-01  9.99678472e-01  1.00071672e+00  9.99630951e-01
  1.00061420e+00  6.99341758e-05  1.70109779e-04 -1.37022125e-04]
[ 1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00


[9.99893474e-01 9.99813793e-01 9.99723228e-01 1.00064773e+00
 9.99502450e-01 2.48554008e-04]
[ 1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
  1.00000000e+00 -2.91433544e-16]
50
[0.86542025 0.85821785 0.93770301 1.00903873 0.91488895 0.02678363
 0.00667201 0.03793246]
[0.93481668 1.04598289 0.98101068 1.05093213 0.99162236 0.01990852
 0.09903173 0.07061951]
[ 9.99902925e-01  1.00082376e+00  1.00007353e+00  9.99651071e-01
  9.99439135e-01  7.52160480e-04 -2.64711067e-05 -4.87633099e-04]
[ 1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
  1.00000000e+00  8.60422844e-16  3.95516953e-16 -1.53523028e-16]
51
[0.90921028 0.94799916 0.81526955 0.8297206  0.82220074]
[1.04940475 0.9467682  0.90146777 0.9835054  1.0608517 ]
[0.99955508 1.00054012 1.00003835 1.00042397 0.99985063]
[1. 1. 1. 1. 1.]
52
[0.71759489 0.9588348  1.08971702 0.77372527 0.87296076 0.01733285]
[0.98422182 1.0079005  1.10983524 1.03976581 0.92014521 0.11476135]
[1.00045095e+00 9.99970206e-01 

75
[ 0.86424486  0.99680228  0.7074497   0.92443186  0.82826723 -0.01558233
 -0.06221881 -0.14289178]
[ 1.02064659  1.03317225  0.98030033  1.01677321  1.00339872 -0.04525397
 -0.12530027  0.01787424]
[ 9.99702398e-01  9.99960943e-01  9.99997718e-01  9.99794052e-01
  9.99882107e-01 -4.62939207e-05  1.17507863e-04  5.91152627e-04]
[ 1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
  1.00000000e+00 -2.89698820e-16  1.17961196e-16 -8.32667268e-17]
76
[ 0.73314416  1.0207972   0.82987462  0.99098328  0.85907378  0.00427274
 -0.03239563]
[ 0.95025271  1.02582187  1.0551257   0.99762883  0.99640709  0.00837781
 -0.00955158]
[ 9.99741356e-01  9.99897444e-01  1.00022772e+00  9.99083434e-01
  1.00071297e+00 -6.11508650e-04  3.15157802e-04]
[ 1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
  1.00000000e+00 -1.17961196e-16 -4.35415592e-16]
77
[0.80189295 0.96767363 1.02171151 0.8234401  0.81673222 0.00270348
 0.00263825]
[0.99261451 0.99345927 1.00863282 1.01871587 1.

98
[ 0.95869604  0.90951241  0.62810436  1.08339997  0.72526824 -0.0208014 ]
[1.01211497 0.9393131  1.01232191 1.02375717 1.1212465  0.01229929]
[ 9.99971365e-01  9.99945363e-01  1.00043512e+00  9.99694073e-01
  1.00059498e+00 -2.66286746e-04]
[ 1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
  1.00000000e+00 -3.19796273e-15]
99
[0.93253556 0.72914708 0.9278873  0.93248273 0.9859777  0.00912713]
[0.97535817 1.00750168 1.05769158 1.00024259 1.03652258 0.05452166]
[1.00063731e+00 9.99835323e-01 1.00032935e+00 1.00046127e+00
 9.99944026e-01 6.70010995e-04]
[ 1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
  1.00000000e+00 -3.33066907e-16]
100
[0.77888913 0.99835578 0.61305477 1.15222459 0.51495016]
[1.04077939 1.03176217 0.98405427 1.02430272 0.9915243 ]
[0.99971808 0.99979908 1.00031843 0.99993611 1.00011082]
[1. 1. 1. 1. 1.]
101
[ 0.65977763  0.88087963  0.75084361  0.93648937  0.87134203 -0.01614485]
[ 0.97415766  1.04106306  0.9394898   0.94013941  1.0864

...

383
[ 0.73748967  1.10336004  0.83880202  0.7193061   1.16478755 -0.0230919
  0.00998568]
[ 1.11229118  1.02422135  0.96321489  0.97701534  1.04372007 -0.12131786
 -0.01416884]
[ 1.00030852e+00  1.00023401e+00  9.99571344e-01  1.00008412e+00
  1.00011937e+00 -3.43443739e-04  6.39157665e-05]
[1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 2.55264559e-15 2.63677968e-16]
384
[0.65564421 0.86932232 1.15334758 0.91946033 0.74381879]
[1.01340761 0.9419477  1.03632741 1.03921711 1.00671858]
[0.99963047 1.00001073 1.00017784 1.00008127 1.00029659]
[1. 1. 1. 1. 1.]
385
[ 0.6980101   1.0187715   0.83666817  0.91283143  0.82933962 -0.00587056]
[ 1.00668613  1.0189859   0.96031022  1.01700216  0.98293588 -0.08057331]
[1.00024076e+00 9.99830066e-01 1.00046405e+00 9.99593900e-01
 1.00031784e+00 1.87449056e-04]
[ 1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
  1.00000000e+00 -2.49800181e-16]
386
[ 0.76809482  1.02409891  0.67041566  0.96586942  0.99274899

[ 9.99727656e-01  1.00000672e+00  9.99768848e-01  1.00030128e+00
  9.99855523e-01 -7.34031833e-04 -4.08772840e-05]
[ 1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
  1.00000000e+00  8.32667268e-17 -3.05311332e-16]
409
[0.7152063  0.83282061 1.05227055 0.86146122 0.89610454]
[0.97029076 1.09223893 0.97140712 0.99532352 1.01573077]
[0.99958124 0.99933968 1.00020899 1.00045931 0.99954686]
[1. 1. 1. 1. 1.]
410
[1.06434441 0.84529385 0.81417539 1.006053   0.76960369]
[1.06433664 0.96013171 0.99573767 0.99749638 1.01920444]
[0.99985894 1.00033019 0.99983265 0.99995089 0.99991535]
[1. 1. 1. 1. 1.]
411
[ 0.73136996  1.07639915  0.78830223  0.74980321  0.96233016 -0.04601205]
[ 0.901327    1.05594416  1.00238511  0.97082263  1.06794    -0.10510692]
[1.00037114e+00 1.00003261e+00 9.99703631e-01 1.00012719e+00
 9.99846976e-01 2.83798370e-04]
[1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 5.82867088e-16]
412
[0.73581136 1.31145671 0.53899392 0.9820361 

[9.99433904e-01 1.00098074e+00 1.00014681e+00 9.99937396e-01
 9.99869366e-01 1.35246777e-04]
[1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.11022302e-16]
436
[0.67813562 1.07859111 1.07338901 0.78192286 0.64389859 0.02585158]
[1.03186399 0.95009189 1.02190566 0.99001211 1.0179818  0.07984314]
[9.99396360e-01 1.00068823e+00 1.00015279e+00 9.99616199e-01
 9.99972944e-01 6.81429442e-04]
[ 1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
  1.00000000e+00 -2.77555756e-16]
437
[ 0.57020991  0.95423454  0.97589804  0.73361605  0.65808429 -0.00246573
  0.08313177  0.02269847]
[0.91812812 0.96999973 0.99368205 0.92198535 1.04994528 0.02200419
 0.09891698 0.06428961]
[ 1.00062943e+00  9.99829704e-01  1.00035661e+00  1.00009639e+00
  9.99814019e-01  3.52819900e-05  4.28419369e-04 -3.53458870e-04]
[1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 4.49293380e-16 5.55111512e-16 5.89805982e-16]
438
[0.58863799 1.20709345 0.92467

[1.00050397e+00 1.00002931e+00 1.00031503e+00 9.99869077e-01
 1.00009017e+00 2.29767101e-04]
[ 1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
  1.00000000e+00 -1.94289029e-16]
462
[0.92083673 0.9843843  0.71822472 0.67703998 1.09672233 0.06164954
 0.00759177]
[1.07162343 0.9983732  0.9781707  1.15217198 0.85501528 0.01153971
 0.02812425]
[ 1.00047177e+00  9.99901793e-01  1.00023741e+00  1.00005764e+00
  9.99924326e-01 -3.63327841e-04 -1.59431587e-04]
[ 1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
  1.00000000e+00 -1.45716772e-16 -1.17961196e-16]
463
[ 0.80700593  0.65126645  0.92470621  1.24965117  0.6838339  -0.00520826
 -0.01328619  0.01001685]
[0.99515415 0.89792667 1.08311973 1.07830506 0.93878601 0.06502105
 0.0519425  0.04183414]
[ 1.00029563e+00  1.00024516e+00  1.00007396e+00  9.99404110e-01
  1.00052369e+00  8.75601255e-05 -5.21395791e-04  7.78516802e-04]
[1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 3.27862737e-

488
[0.79223264 0.80932579 0.68636582 1.39397863 0.29120943 0.20105921]
[ 1.06150803  0.95113952  1.06103123  0.98538098  1.01149297 -0.02823471]
[ 1.00016959e+00  9.99653809e-01  9.99821507e-01  1.00079210e+00
  9.99220600e-01 -1.25981100e-04]
[1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 3.74700271e-16]
489
[0.71749336 1.08448378 0.9183046  0.93460337 0.83308281 0.03252926
 0.00881038]
[ 1.00748075  1.00922638  1.03268251  1.08351502  0.9555375   0.06005639
 -0.00241688]
[1.00008698e+00 1.00001256e+00 9.99841194e-01 1.00004673e+00
 9.99860026e-01 3.48612397e-04 1.30746679e-05]
[1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.24900090e-16 1.38777878e-17]
490
[0.46689767 1.28739848 0.82042855 0.9810561  0.80820233 0.04963969
 0.01625386]
[0.91393754 1.07444441 1.03552746 0.93072975 1.07017917 0.01757552
 0.13319553]
[ 1.00048543e+00  1.00012250e+00  1.00014384e+00  1.00018340e+00
  1.00006822e+00 -4.41733613e-04  5.60826677e-

In [5]:
# result summary
results = pd.DataFrame({'exact': [np.mean(coverage_exact1), np.mean(coverage_exact2), np.mean(length_exact)], 
                        'approx': [np.mean(coverage_approx1), np.mean(coverage_approx2), np.mean(length_approx)]},
                       index=['Coverage rate (target 1)', 'Coverage rate (target 2)', 'CI length']).round(4) 
results

,exact,approx
Coverage rate (target 1),0.8933,0.8958
Coverage rate (target 2),0.8934,0.8955
CI length,0.1999,0.1981


# Linear heterogeneous model

In [6]:
# # model setting
# num_set = []
# coverage_exact = []
# coverage_approx = []
# length_exact = []
# length_approx = []
#
# n, p = 500, 200
# mu, Sig = np.zeros(p), cov_generate(np.ones(p), 0.5)
# beta = np.zeros(p)
# beta[0:5] = 1
# tau = 0.75